In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Madhav\\Documents\\NLP\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [20]:
%pwd

'c:\\Users\\Madhav\\Documents\\NLP\\Text-Summarizer'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [22]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingesiton_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingesiton_config

In [24]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        extracts the zip file into the directory
        function returns none"""
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [19]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-12-04 12:17:12,955: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-04 12:17:12,955: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-04 12:17:12,963: INFO: common: Created directory at : artifacts]
[2023-12-04 12:17:12,965: INFO: common: Created directory at : artifacts/data_ingestion]
[2023-12-04 12:17:17,079: INFO: 3071415012: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 61EC:113B:EEF273:12C8BA8:656D75F5
Accept-Ranges: bytes
Date: Mon, 04 Dec 2023 06:47:19 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10244-MAA
X-Cach